In [31]:
!pip install statsmodels pmdarima

  Obtaining dependency information for pmdarima from https://files.pythonhosted.org/packages/31/d5/871fc78ddd59c44660c6438c9eb5a164c20b11a5ad72ecbab07756d29cd1/pmdarima-2.0.4-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_28_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 1.4 MB/s eta 0:00:00m eta 0:00:010:00:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.8/233.8 kB 1.9 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 1.9 MB/s eta 0:00:00m eta 0:00:010:00:01
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: patsy
    Found existing installation: patsy 0.5.1
    Uninstalling pat

In [89]:
import pandas as pd
import numpy as np
import os
import random

from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [2]:
barcelona_domestic = "data/barcelona_Domestic_Cup__new.csv"
barcelona_df = pd.read_csv(barcelona_domestic, index_col=0)
barcelona_df

,Season,Squad,Country,Comp,LgRank,MP,W,D,L,GF,GA,GD,Pts,Attendance,Top Team Scorer,Goalkeeper,Notes
0,2022-2023,Barcelona,es ESP,Copa del Rey,SF,5,4,0,1,11,7,4,12,77256.0,"Robert Lewandowski, Ousmane Dembélé... - 2",Marc-André ter Stegen,NaN
1,2021-2022,Barcelona,es ESP,Copa del Rey,R16,2,1,0,1,4,4,0,3,NaN,"Jutglà, Ousmane Dembélé... - 1",Marc-André ter Stegen,NaN
2,2020-2021,Barcelona,es ESP,Copa del Rey,1st,6,5,0,1,16,6,10,15,NaN,"Lionel Messi, Antoine Griezmann... - 3",Marc-André ter Stegen,NaN
3,2019-2020,Barcelona,es ESP,Copa del Rey,5th,3,2,0,1,7,2,5,6,43216.0,Antoine Griezmann - 3,Marc-André ter Stegen,NaN
4,2018-2019,Barcelona,es ESP,Copa del Rey,2nd,9,5,1,3,20,9,11,16,10710.0,"Philippe Coutinho, Luis Suárez... - 3",Jasper Cillessen,NaN
5,2017-2018,Barcelona,es ESP,Copa del Rey,1st,9,7,1,1,24,2,22,22,14752.0,Luis Suárez - 5,Jasper Cillessen,NaN
6,2016-2017,Barcelona,es ESP,Copa del Rey,1st,9,6,2,1,24,9,15,20,62355.0,Lionel Messi - 5,Jasper Cillessen,NaN
7,2015-2016,Barcelona,es ESP,Copa del Rey,1st,9,7,2,0,27,5,22,23,15333.0,"Luis Suárez, Lionel Messi... - 5",Marc-André ter Stegen,NaN
8,2014-2015,Barcelona,es ESP,Copa del Rey,1st,9,9,0,0,34,6,28,27,6775.0,Neymar - 7,Marc-André ter Stegen,NaN


# TODO:
domestic leagues:
- ✅Season,
- ✅Squad,
- Comp — лишити лише цифру, закодувати серед різних файлів
- LgRank - забрати закінчення
- ✅MP, W,D,L,GF,GA,GD,Pts,Attendance - int
- ✅Top Team Scorer - розбити на 2 колонки
- ✅Goalkeeper

- LgRank — погуглити і відповідно до того закодувати
- Seasons by Seasons — не юзати файл
- Attendance - fill using other methods(?)

In [61]:
def concatenate_csvs(directory, csv_type):
    all_files = os.listdir(directory)

    # Filter only the CSV files that end with "Domestic_Cup__new.csv"
    csv_files = [file for file in all_files if not file.endswith(csv_type)]

    # Initialize an empty DataFrame to store the concatenated data
    concatenated_df = pd.DataFrame()

    # Loop through each CSV file and concatenate its data to the DataFrame
    for csv_file in csv_files:
        file_path = os.path.join(directory, csv_file)
        df = pd.read_csv(file_path, index_col=0)
        concatenated_df = pd.concat([concatenated_df, df], ignore_index=True)
    return concatenated_df.dropna(how='all').reset_index(drop=True)

df = concatenate_csvs('data', 'Season-by-Season__new.csv')
df

,Season,Squad,Country,Comp,LgRank,MP,W,D,L,GF,GA,GD,Pts,Attendance,Top Team Scorer,Goalkeeper,Notes
0,2022-2023,Barcelona,es ESP,Copa del Rey,SF,5.0,4.0,0.0,1.0,11.0,7.0,4.0,12.0,77256.0,"Robert Lewandowski, Ousmane Dembélé... - 2",Marc-André ter Stegen,NaN
1,2021-2022,Barcelona,es ESP,Copa del Rey,R16,2.0,1.0,0.0,1.0,4.0,4.0,0.0,3.0,NaN,"Jutglà, Ousmane Dembélé... - 1",Marc-André ter Stegen,NaN
2,2020-2021,Barcelona,es ESP,Copa del Rey,1st,6.0,5.0,0.0,1.0,16.0,6.0,10.0,15.0,NaN,"Lionel Messi, Antoine Griezmann... - 3",Marc-André ter Stegen,NaN
3,2019-2020,Barcelona,es ESP,Copa del Rey,5th,3.0,2.0,0.0,1.0,7.0,2.0,5.0,6.0,43216.0,Antoine Griezmann - 3,Marc-André ter Stegen,NaN
4,2018-2019,Barcelona,es ESP,Copa del Rey,2nd,9.0,5.0,1.0,3.0,20.0,9.0,11.0,16.0,10710.0,"Philippe Coutinho, Luis Suárez... - 3",Jasper Cillessen,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548,2018-2019,Sevilla,es ESP,Copa del Rey,5th,6.0,3.0,1.0,2.0,7.0,8.0,-1.0,10.0,6636.0,"André Silva, Wissam Ben Yedder - 2",Juan Soriano,NaN
549,2017-2018,Sevilla,es ESP,Copa del Rey,2nd,9.0,7.0,1.0,1.0,19.0,9.0,10.0,22.0,5878.0,Joaquín Correa - 5,Sergio Rico,NaN
550,2016-2017,Sevilla,es ESP,Copa del Rey,9th,4.0,2.0,1.0,1.0,17.0,8.0,9.0,7.0,23937.0,Wissam Ben Yedder - 5,David Soria,NaN
551,2015-2016,Sevilla,es ESP,Copa del Rey,2nd,9.0,7.0,1.0,1.0,22.0,4.0,18.0,22.0,3667.0,Michael Krohn-Dehli - 3,Sergio Rico,NaN


In [101]:
def fill_missing_with_auto_sarima(series):
    # Use auto_arima to automatically select ARIMA hyperparameters
#     arima_model = auto_arima(series[~series.isna()], suppress_warnings=True)

#     # Get the best ARIMA hyperparameters
#     order = arima_model.get_params()['order']
#     seasonal_order = arima_model.get_params()['seasonal_order']

#     # Create SARIMA model
#     model = SARIMAX(series, order=order, seasonal_order=seasonal_order)
#     results = model.fit(disp=False)

#     # Predict missing values
#     predicted_values = results.get_prediction(start=0, end=len(series) - 1)
#     series_filled = series.combine_first(predicted_values.predicted_mean)

    series_filled = series.bfill().ffill()

    return series_filled

def preprocess_lg_rank(df):
    """
    'QF' -- 6
    'SF' -- 3.5
    'F' -- 1.5
    W -- 1
    GR -- 48
    GR1
    'Rd1' 
    'Rd4'
    'Rd2'
    'Rd3'
    'KO'
    'R16' -- 12
    'R32' -- 24
    """
    def delete_endings(value):
        if isinstance(value, str) and (value.endswith('th') or value.endswith('rd') or value.endswith('nd') or value.endswith('st')):
            return int(value[:-2])
        return value
    df['LgRank'] = df['LgRank'].apply(delete_endings)
    rank_dict = {
        'W': 1,
        'F': 1.5,
        'SF': 3.5,
        'QF': 6,
        'R16': 12,
        'R32': 24,
        'GR': 48
    }
    unknown_values = ['GR1', 'Rd1', 'Rd4', 'Rd2', 'Rd3', 'KO']
    random.shuffle(unknown_values)
    current_rank = 48
    for el in unknown_values:
        current_rank *= 1.25
        rank_dict[el] = current_rank
    
    df["LgRank"] = df["LgRank"].map(rank_dict).fillna(df["LgRank"])
    return df


def preprocess_attendance(df):
    filled_df = df.copy()
    filled_df['Attendance'] = filled_df.groupby(['Squad', 'Comp'])['Attendance'].\
        transform(fill_missing_with_auto_sarima)
    # there are NaN which shouldn't be present but appear because for some competitions there are no data for some squads at all
    # could be fixed if we assume that attendance can be modeled between squads or competition or whatever Vlad wants
    return filled_df


def preprocess_top_team_scorer(df):
    def extract_names_and_number(row):
        if pd.isna(row):
            return np.nan, np.nan # maybe should fill nan differently here
        parts = row.split(' - ')
        names = [name.strip().strip('...') for name in parts[0].split(',')]
        number = int(parts[1])
        return names, number

    df[['Top Team Scorers', 'Top Team Goals']] = df['Top Team Scorer'].apply(extract_names_and_number).apply(pd.Series)
    df = df.drop('Top Team Scorer', axis=1)
    return df

def preprocess_goalkeeper(df):
    def extract_names(row):
        if pd.isna(row):
            return np.nan
        names = [name.strip().strip('...') for name in row.split(',')]
        return names

    df['Goalkeeper'] = df['Goalkeeper'].apply(extract_names)
    return df
    

def preprocess(df):
    
    df = df.drop(columns=["Country", "Notes"])
    
    def extract_start_year(year_range):
        return int(year_range.split('-')[0])
    df['Season'] = df['Season'].apply(extract_start_year)
    
    squad_dict_flipped = dict(enumerate(sorted(df['Squad'].unique())))
    squad_dict = {v: k for k, v in squad_dict_flipped.items()}
    df['Squad'] = df['Squad'].map(squad_dict)
    
    df = df[df["Comp"] != "3. Conf Lg"].reset_index(drop=True) # the league is new and there is only one entry
    comp_significance = {
        "1. La Liga": 0,
        "Super Cup": 1,
        "1. European Cup": 2, "1. Champions Lg": 2,
        "2. Europa Lg": 3, "2. UEFA Cup": 3, "3. UEFA Cup": 3,
        "Supercopa de España": 4,
        "Copa del Rey": 5,
        "2. Segunda División": 6
    }
    df["Comp"] = df["Comp"].map(comp_significance)
    
    df = preprocess_lg_rank(df)    
    

    
    df = preprocess_attendance(df)
    df = preprocess_top_team_scorer(df)
    df = preprocess_goalkeeper(df)


    return df

df_pr = preprocess(df)
df_pr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 552 entries, 0 to 551
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Season            552 non-null    int64  
 1   Squad             552 non-null    int64  
 2   Comp              552 non-null    int64  
 3   LgRank            543 non-null    float64
 4   MP                549 non-null    float64
 5   W                 549 non-null    float64
 6   D                 549 non-null    float64
 7   L                 549 non-null    float64
 8   GF                546 non-null    float64
 9   GA                546 non-null    float64
 10  GD                546 non-null    float64
 11  Pts               549 non-null    float64
 12  Attendance        533 non-null    float64
 13  Goalkeeper        428 non-null    object 
 14  Top Team Scorers  533 non-null    object 
 15  Top Team Goals    533 non-null    float64
dtypes: float64(11), int64(3), object(2)
memory u

In [102]:
df_pr

,Season,Squad,Comp,LgRank,MP,W,D,L,GF,GA,GD,Pts,Attendance,Goalkeeper,Top Team Scorers,Top Team Goals
0,2022,1,5,3.5,5.0,4.0,0.0,1.0,11.0,7.0,4.0,12.0,77256.0,[Marc-André ter Stegen],"[Robert Lewandowski, Ousmane Dembélé]",2.0
1,2021,1,5,12.0,2.0,1.0,0.0,1.0,4.0,4.0,0.0,3.0,43216.0,[Marc-André ter Stegen],"[Jutglà, Ousmane Dembélé]",1.0
2,2020,1,5,1.0,6.0,5.0,0.0,1.0,16.0,6.0,10.0,15.0,43216.0,[Marc-André ter Stegen],"[Lionel Messi, Antoine Griezmann]",3.0
3,2019,1,5,5.0,3.0,2.0,0.0,1.0,7.0,2.0,5.0,6.0,43216.0,[Marc-André ter Stegen],[Antoine Griezmann],3.0
4,2018,1,5,2.0,9.0,5.0,1.0,3.0,20.0,9.0,11.0,16.0,10710.0,[Jasper Cillessen],"[Philippe Coutinho, Luis Suárez]",3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
547,2018,5,5,5.0,6.0,3.0,1.0,2.0,7.0,8.0,-1.0,10.0,6636.0,[Juan Soriano],"[André Silva, Wissam Ben Yedder]",2.0
548,2017,5,5,2.0,9.0,7.0,1.0,1.0,19.0,9.0,10.0,22.0,5878.0,[Sergio Rico],[Joaquín Correa],5.0
549,2016,5,5,9.0,4.0,2.0,1.0,1.0,17.0,8.0,9.0,7.0,23937.0,[David Soria],[Wissam Ben Yedder],5.0
550,2015,5,5,2.0,9.0,7.0,1.0,1.0,22.0,4.0,18.0,22.0,3667.0,[Sergio Rico],[Michael Krohn-Dehli],3.0


In [104]:
df_pr.to_csv("preprocessed_football.csv", index=False)